# STATS790 Project

In [1]:
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
import math
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from itertools import product
from sklearn.metrics import roc_curve, auc
from scipy.special import expit
import itertools

## CART (Regression tree)

I implemented the Classification and Regression Tree (CART) algorithm based on this blog  (ref: https://insidelearningmachines.com/build-a-decision-tree-in-python/). Implementing CART using object-oriented programming (OOP) was a logical choice, as it provides a clear and easily understandable structure. In my implementation, I created two classes: one for nodes and the other for the tree, which essentially comprises of nodes. The node class represents the decision points in the tree, and the tree class encapsulates the entire decision-making process. This OOP-based implementation of CART resulted in a highly organized and readable codebase, making it easy to modify and extend as needed. Additionally, I have added a leaf node ID system. This system assigns a unique ID to every newly created leaf node, making it easy to access individual nodes. This feature offers significant benefits to our gradient boosting algorithm as it allows us to access the leaf nodes quickly and efficiently. With this enhancement, our algorithm can operate at a higher level of performance, providing faster and more accurate results.
 
The default CART implementation in scikit-learn cannot handle missing values, and traditional CART algorithms use surrogate splits to handle missing values. However, surrogate splits are computationally expensive and do not guarantee an improvement in effectiveness, as there may not be similar features to replace the missing values. To overcome this limitation, we adopted the naive approach used in xgboost and lightGBM, which directly assigns missing values to nodes with lower sum square error.

Since CART is a binary tree, each split only considers two nodes. If there are n missing values, there are 2^n ways to assign them to these two nodes. For each missing value, there are two options: either it goes into the first node or the second node. Thus, the total number of ways to assign the n missing values is the product of the number of options for each value, which is 2 * 2 * ... * 2 (n times), or simply $2^n$. In our implementation, we simply iterate all possibile combinations and find the best split.

In [3]:
class Node(object):
    '''
    Class of tree node
    '''

    def __init__(self, split_column = None, split_value = None, leaf_value = None, depth = None):
        self.split_column = split_column  # split feature (None if the node is leaf node)
        self.split_value = split_value    # split value in the current split feature (None if the node is leaf node)
        self.leaf_value = leaf_value      # leaf value (None if the node is decision node)
        self.depth = depth                # depth of the node
        self.left = None                  # left child of the decision node (None if the node is leaf node)
        self.right = None                 # right child of the decision node (None if the node is leaf node)
        self.data = None                  # data points assigned to the leaf node (None if the node is decision node)
        self.leaf_id = None               # leaf node ID (i.e 1,2,3...n) (None if the node is decision node)
        
    def update_parameters(self,split_column,split_value):
        '''
        Function to update the split feature and split value of the node 

        Inputs:
            split_column - split feature
            split_value  - split value in the current split feature
        '''
        self.split_column = split_column
        self.split_value = split_value

    def get_parameters(self):
        '''
        Function to get parameters of the node 

        Outputs:
            split_column - split feature
            split_value  - split value in the current split feature
        '''
        return self.split_column, self.split_value
    
    def set_children(self, left_child, right_child):
        '''
        Function to set parameters of the node 

        Inputs:
            left_child - left child of the decision node 
            split_value  - right child of the decision node 
        '''
        self.left = left_child
        self.right = right_child

    def get_left_child(self):
        '''
        Function to get left child of the node 

        Outputs:
            left_child - left child of the decision node 
        '''
        return self.left
    
    def get_right_child(self):
        '''
        Function to get right child of the node 

        Outputs:
            right_child - right child of the decision node 
        '''
        return self.right
    


In [4]:
class Tree:
    '''
    Class of decision tree
    '''

    def __init__(self, max_depth, min_sample):
        self.max_depth = max_depth     # maximum depth of the tree
        self.min_sample = min_sample   # minimum number of samples of the tree
        self.decisionTree = None       # tree structure (all nodes of the tree)
        self.leaf_nodes = []           # a list records all leaf nodes
        self.current_id = 0            # a vairable used for updating leaf ID (each leaf node have its unique ID start from 0)
        self.ids = []                  # a list records all leaf nodes IDs
        self.leaf_values = []          # a list records all leaf values (for fast access)
    
    
    def sse(self, data):
        '''
        Sum of square error used for spliting node

        Inputs:
            data - current dataset
        Ouputs:
            sum of square error
        '''
        return np.sum((data[:,-1] - np.mean(data[:,-1]))**2)
        
    def leaf_value(self,data):
        '''
        Function to compute the leaf value of each leaf node (Mean of all response varaibles of the current dataset)

        Inputs:
            data - current dataset
        Ouputs:
            leaf value
        '''
        return np.mean(data[:,-1])
    
    def grow_tree(self, node, data, depth = 0):
        '''
        Recursive function to grow the tree                                 

        Inputs:
            node  - current node
            data  - current dataset
            depth - current depth of the tree (default start from 0)

        '''

        # If the not satisfy leaf node condition:
        #  1. not reach max depth. 
        #  2. number of data points in current node lager than the threshold
        # then keep spliting
        if depth < self.max_depth - 1 and data.shape[0] > self.min_sample and len(np.unique(data[:,-1])) != 1:
            num_features = data.shape[1] - 1  # Compute number of features
            split_column = None                
            split_value = None
            left_data = None      # initialize left subset of data after split 
            right_data = None     # initialize right subset of data after split 
            split_Error = np.inf  # initialize total error after split
            # looping all possible features/values
            for feature in range(num_features):
                for value in np.unique(data[:,feature]):
                    # split the dataset into left and right based on current features/values
                    left_indices = data[:, feature] <= value  
                    right_indices = data[:, feature] > value
                    left_data_temp = data[left_indices]
                    right_data_temp = data[right_indices]
                    # make sure the number of datapints in each node after split satisfy the threshold
                    if left_data_temp.shape[0] >= self.min_sample and right_data_temp.shape[0] >= self.min_sample:
                        # compute the error (impurity)
                        err = self.sse(left_data_temp) + self.sse(right_data_temp)
                        # update the error, split feature/value if it is better
                        if err < split_Error:
                            split_column = feature
                            split_value = value
                            split_Error = err
                            left_data = left_data_temp
                            right_data = right_data_temp
            # If split successfully
            if split_column is not None:
                left_node = Node()   # Create new left child node object
                right_node = Node()  # Create new right child node object
                # Update parameters and connect left and right child to the current node
                node.update_parameters(split_column,split_value)
                node.set_children(left_node, right_node)
                # Keep grow tree recursively for using left and right data after split 
                self.grow_tree(node.get_left_child(),left_data,depth+1)
                self.grow_tree(node.get_right_child(),right_data,depth+1)
            # If split not successfully, then set the node as leaf node
            else: 
                # Make current node as leaf node
                node.leaf_value = self.leaf_value(data)  # Compute the leaf value
                node.data = data   # assign the data points belong to it                
                node.leaf_id = self.current_id  # set the leaf ID
                self.ids.append(self.current_id)  
                self.leaf_values.append(node.leaf_value)
                self.current_id += 1  # Update next ID
                self.leaf_nodes.append(node)
                return
        # If reach the maximum depth, or number of data points in current node smaller or equal than the threshold
        else:
            # Make current node as leaf node
            node.leaf_value = self.leaf_value(data)
            node.data = data
            node.leaf_id = self.current_id
            self.ids.append(self.current_id)
            self.leaf_values.append(node.leaf_value)
            self.current_id += 1
            self.leaf_nodes.append(node)
            return

    def grow_tree_missing(self, node, data, depth = 0):
        '''
        Recursive function to grow the tree specially used for handling dataset with missing value                                

        Inputs:
            node  - current node
            data  - current dataset
            depth - current depth of the tree (default start from 0)

        '''

        # If the not satisfy leaf node condition:
        #  1. not reach max depth. 
        #  2. number of data points in current node lager than the threshold
        # then keep spliting
        if depth < self.max_depth - 1 and data.shape[0] > self.min_sample and len(np.unique(data[:,-1])) != 1:
            num_features = data.shape[1] - 1  # Compute number of features
            split_column = None                
            split_value = None
            left_data = None      # initialize left subset of data after split 
            right_data = None     # initialize right subset of data after split 
            split_Error = np.inf  # initialize total error after split
            # looping all possible features/values
            for feature in range(num_features):
                #col_i = data[:,feature] # all feature values (column) include Nan
                #col_i_no_na = col_i[~np.isnan(col_i)]  # filter Nan
                has_nan_indices = np.where(np.isnan(data[:, feature]))[0]
                no_nan_indices = np.where(~np.isnan(data[:, feature]))[0]
                has_nan = data[has_nan_indices, :]
                no_nan = data[no_nan_indices, :]    

                for value in np.unique(no_nan[:,feature]):
                    # split the dataset into left and right based on current features/values
                    left_indices = data[:, feature] <= value  
                    right_indices = data[:, feature] > value
                    left_data_temp = data[left_indices]
                    right_data_temp = data[right_indices]

                    # Generate all possible combinations of assigning the new rows to the two dataframes
                    combos = list(itertools.product([0, 1], repeat=has_nan.shape[0]))
                    # Loop over each combination
                    for combo in combos:
                        # Create a copy of the original dataframes to modify
                        df1_copy = left_data_temp.copy()
                        df2_copy = right_data_temp.copy()

                        # Assign the new rows to df1 or df2 based on the combo
                        for i, row in enumerate(has_nan):
                            if combo[i] == 0:
                                df1_copy = np.vstack([df1_copy, row])
                            else:
                                df2_copy = np.vstack([df2_copy, row])

                        # make sure the number of datapints in each node after split satisfy the threshold
                        if df1_copy.shape[0] >= self.min_sample and df2_copy.shape[0] >= self.min_sample:
                            # compute the error (impurity)
                            err = self.sse(df1_copy) + self.sse(df2_copy)
                            # update the error, split feature/value if it is better
                            if err < split_Error:
                                split_column = feature
                                split_value = value
                                split_Error = err
                                left_data = df1_copy
                                right_data = df2_copy
            # If split successfully
            if split_column is not None:
                left_node = Node()   # Create new left child node object
                right_node = Node()  # Create new right child node object
                # Update parameters and connect left and right child to the current node
                node.update_parameters(split_column,split_value)
                node.set_children(left_node, right_node)
                # Keep grow tree recursively for using left and right data after split 
                self.grow_tree(node.get_left_child(),left_data,depth+1)
                self.grow_tree(node.get_right_child(),right_data,depth+1)
            # If split not successfully, then set the node as leaf node
            else: 
                # Make current node as leaf node
                node.leaf_value = self.leaf_value(data)  # Compute the leaf value
                node.data = data   # assign the data points belong to it                
                node.leaf_id = self.current_id  # set the leaf ID
                self.ids.append(self.current_id)  
                self.leaf_values.append(node.leaf_value)
                self.current_id += 1  # Update next ID
                self.leaf_nodes.append(node)
                return
        # If reach the maximum depth, or number of data points in current node smaller or equal than the threshold
        else:
            # Make current node as leaf node
            node.leaf_value = self.leaf_value(data)
            node.data = data
            node.leaf_id = self.current_id
            self.ids.append(self.current_id)
            self.leaf_values.append(node.leaf_value)
            self.current_id += 1
            self.leaf_nodes.append(node)
            return
        
    def fit(self,X,y):
        '''
        Function to train the data to build the CART                               

        Inputs:
            X  - Predictor variables
            y  - Response variables
        '''

        # Merge Predictor variables and Response variables into one dataset
        data = np.concatenate((X,y.reshape(-1,1)),axis=1)  
        # set the root node of the tree
        self.decisionTree = Node()
        # grow the tree
        self.grow_tree(self.decisionTree,data,0)
    
    def fit_missing(self,X,y):
        '''
        Function to train the data to build the CART specially used for handling dataset with missing value                             

        Inputs:
            X  - Predictor variables
            y  - Response variables
        '''

        # Merge Predictor variables and Response variables into one dataset
        data = np.concatenate((X,y.reshape(-1,1)),axis=1)  
        # set the root node of the tree
        self.decisionTree = Node()
        # grow the tree
        self.grow_tree_missing(self.decisionTree,data,0)
    
    def find_leaf(self,decisionTree,x):
        '''
        Function to find the leaf node that the input data point belong to, and return the predict leaf value                      

        Inputs:
            decisionTree  - the trained CART
            x             - one data point
        Output:
            leaf value
        '''

        # If it is not the leaf node, then traverse the tree recursively (go to child nodes)
        if decisionTree.leaf_value == None:
            split_column, split_value = decisionTree.get_parameters()
            if x[split_column] <= split_value:
                return self.find_leaf(decisionTree.get_left_child(),x)   # go left if smaller or equal than split value
            else:
                return self.find_leaf(decisionTree.get_right_child(),x)  # go right if greater than split value
        # if it is leaf node, return leaf value
        else:
            return decisionTree.leaf_value
    
    def predict(self,X):
        '''
        Prediction function based on the trained CART                     

        Inputs:
            X  - predictor dataset
        Output:
            array of predicted values
        '''

        res = []
        # go through each data points and make prediction
        for i in range(X.shape[0]):
            res.append(self.find_leaf(self.decisionTree,X[i]))
        
        return np.array(res).flatten()
    

    def find_leaf_id(self,decisionTree,x):
        '''
        Function to find leaf ID of a data point based on the trained CART                     

        Inputs:
            decisionTree  - the trained CART
            x             - one data point
        Output:
            leaf ID
        '''

        # If it is not the leaf node, then traverse the tree recursively (go to child nodes)
        if decisionTree.leaf_value == None:
            split_column, split_value = decisionTree.get_parameters()
            if x[split_column] <= split_value:
                return self.find_leaf_id(decisionTree.get_left_child(),x)
            else:
                return self.find_leaf_id(decisionTree.get_right_child(),x)
        # if it is leaf node, return leaf value
        else:
            return decisionTree.leaf_id

    def apply(self,X):
        '''
        Function to find all leaf ID of a dataset based on the trained CART                     

        Inputs:
            X  - predictor dataset
        Output:
            array of leaf ID each data point belong to
        '''
        
        res = []
        # go through each data points and make prediction
        for i in range(X.shape[0]):
            res.append(self.find_leaf_id(self.decisionTree,X[i]))
        
        return np.array(res).flatten()

### Artificial regression dataset
In this section, we create an artificial regression dataset to evaluate and compare the performance of both our implemented CART model and the CART model from scikit-learn. I test them on 10 random regression dataset, we can see the performance are similar. The CART scikit-learn are slightly better thanour implemented CART.

In [46]:
import random
random_seeds = [random.randint(0, 10000) for i in range(10)]  # generate 10 random seeds

# container to store the RMSE from our implemented CART
rmse_custom = []
# container to store the RMSE from scikit-learn CART
rmse_scikit = []

for i in random_seeds:
    # generate regression dataset for random seed i, and do train/test split
    X, y = make_regression(n_samples=100, n_features=10, random_state=i)
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.3)

    # train and predict our implemented CART
    rgt = Tree(max_depth=5, min_sample = 3)
    rgt.fit(X_train_reg,y_train_reg)
    y_pred_reg = rgt.predict(X_test_reg)
    rmse_custom.append(np.sqrt(mean_squared_error(y_test_reg,y_pred_reg)))  # append to list

    # train and predict using CART from scikit learn
    rgt_scikit = DecisionTreeRegressor(max_depth=5, min_samples_split = 3, min_samples_leaf = 3)
    rgt_scikit.fit(X_train_reg,y_train_reg)
    y_pred_reg_scikit = rgt_scikit.predict(X_test_reg)
    rmse_scikit.append(np.sqrt(mean_squared_error(y_test_reg,y_pred_reg_scikit)))  # append to list

print("RMSE for our implemented CART in 10 random dataset " + str(rmse_custom))
print("RMSE for scikit-learn CART in 10 random dataset " + str(rmse_scikit))

RMSE for our implemented CART in 10 random dataset [196.87157998975198, 206.79586271247052, 154.5985378655847, 120.73831934706669, 169.22009224003475, 198.79538375320283, 166.2302440144937, 125.31812261226318, 158.39924279843459, 181.20175540652903]
RMSE for scikit-learn CART in 10 random dataset [208.85739802125946, 191.40829984128482, 140.46410584408977, 115.69282443677845, 177.5959540784621, 216.47153238147644, 177.39888598042444, 133.10915197915398, 146.43188358955544, 191.93575290183048]


## Gradient Boosting Decision Tree (GBDT)

I implemented Gradient Boosting Decision Tree (GBDT) for binary classification based on this blog (reference:https://towardsdatascience.com/all-you-need-to-know-about-gradient-boosting-algorithm-part-2-classification-d3ed8f56541e). Additionally, I also implemented GBDT for multiclass classificaiton. 

In [5]:
class gradient_boosting(object):
    '''
    Class of gradient boosting decision tree (GBDT)
    '''

    def __init__(self,M,base_model="CART",max_depth = 1,min_sample = 1,learning_rate = 0.1, method = "binary classification",loss = "logistic", missing_value = False,tol = None):
        self.M = M                           # number of trees
        self.base_model = base_model         # base classifier (default is CART)
        self.max_depth = max_depth           # maximum depth of the base classifier (max depth for CART)
        self.min_sample = min_sample         # minimum number of sample in a node
        self.learning_rate = learning_rate   # learning rate 
        self.method = method                 # usage of gradient boosting (binary or multiclass classification)
        self.loss = loss                     # loss function
        self.missing_value = missing_value   # If there's missing value in the dataset
        self.tol = tol                       # tolerance
        self.trees = []                      # a list record M trees in generated in gradient boosting
        self.num_class = None                # number of classes (this is only used in multiclass classification)
        self.col_names = None                # column names
    
    def softmax(self,x):
        '''
        Softmax function (used for multiclass classification)                    

        Inputs:
            x  - one dimensional array of data
        Output:
            probability of each element of the array
        '''

        e_x = math.e**(x - np.max(x, axis=1, keepdims=True))
        return e_x / np.sum(e_x, axis=1, keepdims=True)

    def logit(self,x):
        '''
        Logit function (used for binary classification)                    

        Inputs:
            x  - one dimensional array of data
        Output:
            probability of each element of the array
        '''
        return expit(x)   #1/(1 + np.exp(-x))

    def fit(self,x,y):
        '''
        Function to train GBDT                              

        Inputs:
            x  - Predictor variables
            y  - Response variables
        '''

        ##binary classification#############################################################################################

        if self.method == "binary classification":

            self.num_class = 2
            # initial prediction
            F0 = np.log(np.mean(y)/(1-np.mean(y)))
            self.F0 = np.array([F0]*len(y))
            # predict value after mth tree, we make it equal to the initial prediction, it will keep updating when grow a new tree
            fm = self.F0.copy()
    
            # grow M trees
            for m in range(self.M):
                # compute residual
                p = self.logit(fm)
                r_im = y - p
                if self.base_model == "CART":
                    # fit a CART implemented in previous section using residual
                    tree = Tree(max_depth = self.max_depth, min_sample = self.min_sample)  # set max depth and min number of sample for CART
                    if self.missing_value == True:
                        tree.fit_missing(x.values, r_im.values)  # train CART in missing value mode
                    else:
                        tree.fit(x.values, r_im.values)  # train CART in normal mode
                    nodes = tree.apply(x.values)     # find corresponding leaf ID of each data point based on the trained CART
                    
                    # go through all leaf nodes
                    for i in tree.ids:
                        # find subset of data points belong to node i
                        sub = np.equal(i, nodes)
                        #compute the leaf value for node i
                        denominator = (np.sum(p[sub]*(1-p[sub])))
                        if np.abs(denominator) <= 1e-150:
                            gamma = 0
                        else:
                            gamma = (np.sum(r_im[sub]))/denominator
                        # update predict value for all data points belong to node i
                        fm[sub] += self.learning_rate*gamma
                        # update leaf value
                        tree.leaf_values[i] = gamma
                        tree.leaf_nodes[i].leaf_value = gamma
                    # append current tree to the list of tree
                    self.trees.append(tree)

        ##multiclass classification#########################################################################################

        one_hot = None # store one-hot encoding for response variable
        if self.method == "multiclass classification":     
            self.num_class = y.nunique() # count number of classes
            one_hot = pd.get_dummies(y)  # one-hot encoding
            self.col_names = one_hot.columns.tolist()  # get column names
        
            # initial prediction
            F0 = []
            for i in range(self.num_class):
                coli = one_hot.iloc[:,i]   # get column for ith class in one-hot encoding
                num_class_i = coli.value_counts()[1]  # count number of 1's 
                F0.append(np.array([num_class_i/len(y)]*len(y)))  # Calculate the proportion of ith class 
            
            self.F0 = np.array(F0)
            # predict value after mth tree, we make it equal to the initial prediction, it will keep updating when grow a new tree
            fm = self.F0.copy()

            # grow M trees
            for m in range(self.M):
                if self.base_model == "CART":
                    # a list to store k trees for k classes 
                    ktrees = [] 
                    # compute softmax
                    p = self.softmax(fm.T).T
                    # go through all classes and fit CART for the residual
                    for k in range(self.num_class):
                        p_km = p[k] # predict value for kth tree
                        r_km = np.array(one_hot.iloc[:,k]) - p_km # compute residual

                        # fit kth CART tree
                        tree = Tree(max_depth = self.max_depth, min_sample = self.min_sample)  # set max depth and min number of sample for CART
                        if self.missing_value == True:
                            tree.fit_missing(x.values, r_km)      # train CART
                        else:
                            tree.fit(x.values, r_km)      # train CART
                        nodes = tree.apply(x.values)  # find corresponding leaf ID of each data point based on the trained CART

                        # go through all leaf nodes
                        for i in tree.ids:
                            # find subset of data points belong to node i
                            sub = np.equal(i, nodes)
                            
                            #compute the leaf value for node i
                            denominator = (np.sum(np.abs(r_km[sub])*(1-np.abs(r_km[sub]))))
                            if np.abs(denominator) <= 1e-150:   # if the denominator is too small, set gamma equal to 0. This is to avoid dividing zero error
                                gamma = 0
                            else:                            
                                gamma = ((self.num_class-1)/self.num_class) * (np.sum(r_km[sub])/denominator)
                            # update predict value for all data points belong to node i
                            fm[k][sub] += self.learning_rate*gamma
                            # update leaf value
                            tree.leaf_values[i] = gamma
                            tree.leaf_nodes[i].leaf_value = gamma
                        ktrees.append(tree)
                    self.trees.append(ktrees)

    def predict_prob(self,x):
        '''
        Function to predict probabilities based on the GBDT                     

        Inputs:
            X  - predictor dataset
        Output:
            array of predicted probabilities
        '''
        
        if self.method == "binary classification":

            # important! resize the initial prediction as the input size
            Fm = np.resize(self.F0, x.shape[0])
            # go through each tree and adding up the predictions
            for m in range(self.M):
                Fm += self.learning_rate * self.trees[m].predict(x.values)
            # Output the predict probabilities
            prob = self.logit(Fm)
        
            return prob

        if self.method == "multiclass classification":

            # important! resize the initial prediction as the input size
            F0_pred = []
            for i in range(self.num_class):
                F0_pred.append(np.resize(self.F0[i], x.shape[0]))
            Fm = np.array(F0_pred)

            # go through each tree and adding up the predictions
            for m in range(self.M):
                for k in range(self.num_class):
                    Fm[k] += self.learning_rate * self.trees[m][k].predict(x.values)

            # compute the probability
            p = self.softmax(Fm.T).T
            # store the probability in a dataframe
            prob = pd.DataFrame(columns=self.col_names)
            for k in range(self.num_class):
                prob[prob.columns[k]] = p[k]

            return prob


    def predict(self,x):
        '''
        Function to predict labels based on the GBDT                     

        Inputs:
            X  - predictor dataset
        Output:
            array of prediction labels
        '''
        
        if self.method == "binary classification":
            # important! resize the initial prediction as the input size
            Fm = np.resize(self.F0, x.shape[0])
            # go through each tree and adding up the predictions
            for m in range(self.M):
                Fm += self.learning_rate * self.trees[m].predict(x.values)
            # Output the predict probabilities
            prob = self.logit(Fm)

            # return the predict labels
            return (prob > 0.5).astype(int)


        if self.method == "multiclass classification":
            # important! resize the initial prediction as the input size
            F0_pred = []
            for i in range(self.num_class):
                F0_pred.append(np.resize(self.F0[i], x.shape[0]))
            Fm = np.array(F0_pred)

            # go through each tree and adding up the predictions
            for m in range(self.M):
                for k in range(self.num_class):
                    Fm[k] += self.learning_rate * self.trees[m][k].predict(x.values)

            # compute the probability
            p = self.softmax(Fm.T).T
            # store the probability in a dataframe
            prob = pd.DataFrame(columns=self.col_names)
            for k in range(self.num_class):
                prob[prob.columns[k]] = p[k]

            return self.max_col_name(prob)
        

    def max_col_name(self,df):
        '''
        Function to find the class with maximum probability as the prediction label                     

        Inputs:
            df  - probability dataframe generate in predict function
        Output:
            array of prediction labels
        '''
        
        # Create a list to store the column names of the maximum values
        col_names = []

        # Loop through each row in the DataFrame
        for index, row in df.iterrows():
            # Find the name of the column with the maximum value in the row
            max_col = row.idxmax()
            # Append the column name to the list
            col_names.append(max_col)

        # Return the list of column names
        return col_names
    
    

## K-Fold Cross Validation

The following is the k-fold cross-validation function used for our implemented GBDT. Different metrics, such as accuracy, F1 score, and ROC, can be applied. Also, we used this in the later hyperparameter tuning function.

In [6]:
def kfoldcrossGBDT(trainx,trainy,M,max_depth,min_sample,learning_rate,method,kfold=10, missing_value = False, scoring = "acc"):
    '''
    Function to perform K-Fold Cross Validation for GBDT                      

    Inputs:
        trainx        -    predictor variables of training dataset
        trainy        -    response variables of training dataset
        M             -    Number of CART to grow
        max_depth     -    maximum depth of the base classifier (max depth for CART)
        min_sample    -    minimum number of sample in a node
        learning_rate -    learning rate
        method        -    binary or multiclass classification
        kfold         -    Number of folds want to use
        scoring       -    validity index (accuracy, adjusted rand index, f1 score, ROC)
        
    Output:
        average score and its standard deviation
    '''

    kf = KFold(n_splits=kfold) # k splits
    iter1 = kf.split(trainx)

    # store the score for each split
    acc = []
    ari = []
    f1 = []
    roc_auc = []

    while True:
        try:
            # next split:
            i = next(iter1)
            # split data
            train_x = trainx.iloc[i[0]]
            train_y = trainy.iloc[i[0]]
            test_x =  trainx.iloc[i[1]]         
            test_y =  trainy.iloc[i[1]] 
            #reset index
            train_x.reset_index(drop=True, inplace=True)
            test_x.reset_index(drop=True, inplace=True)
            train_y.reset_index(drop=True, inplace=True)
            test_y.reset_index(drop=True, inplace=True)

            #train GBDT and predict
            GBDT = gradient_boosting(M=M,max_depth = max_depth,min_sample = min_sample,learning_rate = learning_rate,method=method,missing_value = missing_value)
            GBDT.fit(train_x,train_y)
            y_pred = GBDT.predict(test_x)
            
            # record the score
            acc.append(accuracy_score(y_pred, test_y))
            ari.append(adjusted_rand_score(y_pred, test_y))
            f1.append(f1_score(y_pred, test_y, average='weighted'))
            if method == "binary classification":
                fpr, tpr, thresholds = roc_curve(test_y, y_pred)
                roc_auc.append(auc(fpr, tpr))
            
        except StopIteration:
            break
    
    if scoring == "acc":
        return [np.mean(acc),np.std(acc)]
    if scoring == "ari":
        return [np.mean(ari),np.std(ari)]
    if scoring == "f1":
        return [np.mean(f1),np.std(f1)]
    if scoring == "roc":
        return [np.mean(roc_auc),np.std(roc_auc)]


## Hyperparameter tunning

In [7]:
def grid_search_cv_GBDT(hyperparams, trainx,trainy, method, kfold=10, scoring = "acc"):
    '''
    Function to do hyperparameter tunning using grid search for GBDT                      

    Inputs:
        hyperparams:  -    dictionary of hyperparameters and their possible values
        trainx        -    predictor variables of training dataset
        trainy        -    response variables of training dataset
        method        -    binary or multiclass classification
        kfold         -    Number of folds want to use
        scoring       -    validity index (accuracy, adjusted rand index, f1 score, ROC)
        
    Output:
        Best parameter set and its score
    '''

    best_params = None  # Best parameter set
    best_score = 0      # Best score

    # Iterate over all combinations of hyperparameters
    for params in product(*hyperparams.values()):

        # Create a dictionary of hyperparameters
        param_dict = dict(zip(hyperparams.keys(), params))
        print("Current parameter set: " + str(param_dict))

        # We have 4 parameters need to tune in our implemented GBDT, get them from the current dictionary
        M = param_dict.get('M')
        max_depth = param_dict.get('max_depth')
        min_sample = param_dict.get('min_sample')
        learning_rate = param_dict.get('learning_rate')

        # compute the cross validation score of current parameter set
        score = kfoldcrossGBDT(trainx,trainy,M ,max_depth,min_sample,learning_rate,method = method,kfold=kfold, scoring = scoring)
        print(str(scoring) + ":" + str(score[0]))

        # If it is better, then update it as the current best parameter set
        if score[0] > best_score:
            best_params = param_dict
            best_score = score[0]

    return best_params, best_score

## Heart Dataset (Binary classification)

This Heat Disease Dataset was compiled in 1988 and comprises four separate databases: Cleveland, Hungary, Switzerland, and Long Beach V. It comprises 76 different attributes, including the predicted attribute, but in all published experiments, only a subset of 14 attributes were utilized. The dataset contains 1025 instances with 9 categorical attributes: Sex, Chest pain type, fasting blood sugar, thal, resting electrocardiographic result, maximum heart rate, exercise-induced angina, the slope, and the number of major vessels, and 4 numerical attributes: Age, Resting blood pressure, serum cholestoral, and ST depression. The "target" field specifically indicates whether a patient has heart disease, with a value of 0 representing the absence of the disease and 1 indicating its presence.

dataset reference: https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset

In [7]:
##Data preprocessing##

df_heart = pd.read_csv('D:\\STATS4T06\\Datasets\\heartdf.csv', index_col=False)
# drop first column which is the ID
df_heart = df_heart.drop(df_heart.columns[0], axis=1)

X = df_heart.drop('target', axis=1)
y = df_heart['target']

# normalize numerical attributes
scaler = MinMaxScaler()
num_cols = ['age','trestbps','chol','thalach','oldpeak']
X[num_cols] = scaler.fit_transform(X[num_cols])

# perform one-hot encoding for categorical attributes
X = pd.get_dummies(X, columns=['slope','ca','thal'])


# perform train-test split
X_train_heart, X_test_heart, y_train_heart, y_test_heart = train_test_split(X, y, test_size=0.3, random_state=4)
# reset index
X_train_heart = X_train_heart.reset_index(drop=True)
X_test_heart = X_test_heart.reset_index(drop=True)
y_train_heart = y_train_heart.reset_index(drop=True)
y_test_heart = y_test_heart.reset_index(drop=True)


In [118]:
##Find best parameter set (Hyperparameter tuning)##
parameters = {'M': [10,50,100], 
              'max_depth': [2,3,6], 
              'min_sample': [2,3,5],
              'learning_rate':[0.1,0.5,1]}
best_params_heart, best_score_heart = grid_search_cv_GBDT(parameters, X_train_heart,y_train_heart, method = "binary classification", kfold=3, scoring = "acc")

Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 2, 'learning_rate': 0.1}
acc:0.8228730822873082
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 2, 'learning_rate': 0.5}
acc:0.8688981868898186
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 2, 'learning_rate': 1}
acc:0.8423988842398885
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 3, 'learning_rate': 0.1}
acc:0.8228730822873082
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 3, 'learning_rate': 0.5}
acc:0.8688981868898186
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 3, 'learning_rate': 1}
acc:0.8423988842398885
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 5, 'learning_rate': 0.1}
acc:0.8228730822873082
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 5, 'learning_rate': 0.5}
acc:0.8688981868898186
Current parameter set: {'M': 10, 'max_depth': 2, 'min_sample': 5, 'learning_rate': 1}
acc:0.8423988842398885
Current

In [119]:
##Print the best parameter set##
print(best_params_heart)
print(best_score_heart)

{'M': 100, 'max_depth': 3, 'min_sample': 5, 'learning_rate': 1}
0.9679218967921895


In [33]:
##Prediction on test set##
gb_heart = gradient_boosting(M=100,max_depth = 3,min_sample = 5,learning_rate = 1,method="binary classification")
gb_heart.fit(X_train_heart,y_train_heart)

# performance evaluation
y_pred_heart_test = gb_heart.predict(X_test_heart)
print("Accuracy: " + str(accuracy_score(y_test_heart, y_pred_heart_test)))
print("F1 score: " + str(f1_score(y_test_heart, y_pred_heart_test, average='weighted')))
fpr, tpr, thresholds = roc_curve(y_test_heart, y_pred_heart_test)
print("ROC_AUC: " + str(auc(fpr, tpr)))
print("============= Classification Report =================")
print(classification_report(y_test_heart, y_pred_heart_test))

Accuracy: 0.9902597402597403
F1 score: 0.9902575829486129
ROC_AUC: 0.9901315789473684
============= Classification Report =================
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       156
           1       1.00      0.98      0.99       152

    accuracy                           0.99       308
   macro avg       0.99      0.99      0.99       308
weighted avg       0.99      0.99      0.99       308



## Credict Approval Dataset (Binary classification)

The Credit Approval Dataset comprises financial and personal information of credit applicants, with 690 instances and 15 attributes. One noteworthy characteristic of this dataset is the diversity of its attributes, including continuous variables, nominal variables with small numbers of values, and nominal variables with larger numbers of values. After removing rows with missing values, 653 instances remain, with 296 instances having a positive credit score and 357 with a negative credit score. The number of numerical and categorical attributes is evenly distributed, with 9 categorical variables (A1, A4, A5, A6, A7, A9, A10, A12, A13) and 4 numerical attributes (A2, A3, A8, A11). All attribute names and values have been replaced with meaningless symbols to preserve the confidentiality of the data. We test the performance of our implemented GBDT for handling missing values using this dataset.

dataset reference: https://archive.ics.uci.edu/ml/datasets/credit+approval

In [39]:
# Data preprocessing
df_ca = pd.read_csv("D:\\STATS4T06\\Datasets\\crx.data", delimiter=",",header=None)
df_ca.columns = ['a0', 'a1', 'a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15'] # set column names
df_ca = df_ca.replace("?", np.nan)  # replace missing value as Nan

##-------------------------Dataset with missing value---------------------------------#
print("Size of dataset removing missing value: " + str(df_ca.shape)) #print size of the dataframe
X_n = df_ca.drop('a15', axis=1)
y_n = df_ca['a15'].map({'+': 1, '-': 0})
# normalize numerical attributes
scaler = MinMaxScaler()
num_cols = ['a1','a2','a7','a10','a13','a14']
X_n[num_cols] = scaler.fit_transform(X_n[num_cols])
# perform one-hot encoding for categorical attributes
X_n = pd.get_dummies(X_n, columns=['a0','a3','a4','a5','a6','a8','a9','a11','a12'])
# train/test split
X_train_ca_n, X_test_ca_n, y_train_ca_n, y_test_ca_n = train_test_split(X_n, y_n, test_size=0.3, random_state=4)

##-------------------------Dataset removing missing value------------------------------# 
df_ca = df_ca.dropna()
print("Size of dataset with missing value: " + str(df_ca.shape)) #print size of the dataframe
X = df_ca.drop('a15', axis=1)
y = df_ca['a15'].map({'+': 1, '-': 0})
# normalize numerical attributes
scaler = MinMaxScaler()
num_cols = ['a1','a2','a7','a10','a13','a14']
X[num_cols] = scaler.fit_transform(X[num_cols])
# perform one-hot encoding for categorical attributes
X = pd.get_dummies(X, columns=['a0','a3','a4','a5','a6','a8','a9','a11','a12'])
# train/test split
X_train_ca, X_test_ca, y_train_ca, y_test_ca = train_test_split(X, y, test_size=0.3, random_state=4)



Size of dataset removing missing value: (690, 16)
Size of dataset with missing value: (653, 16)


We first test the cross validation function that hanlding missing value, we can see the result is very good, but it is much slower than ignoring missing value.

In [53]:
# 5-fold cross validation using dataset with missing value
creditCv = kfoldcrossGBDT(X_train_ca_n,y_train_ca_n,M = 10,max_depth=6,min_sample=5,learning_rate=0.5,method="binary classification",kfold=5, missing_value = True, scoring = "acc")
print("Accuracy and standard deviation: " + str(creditCv))

Accuracy and standard deviation: [0.8654639175257731, 0.023461572512046523]


In [48]:
# 5-fold cross validation using dataset removing missing value
creditCv_no_na = kfoldcrossGBDT(X_train_ca,y_train_ca,M = 10,max_depth=6,min_sample=5,learning_rate=0.5,method="binary classification",kfold=5, missing_value = False, scoring = "acc")
print("Accuracy and standard deviation: " + str(creditCv_no_na))

Accuracy and standard deviation: [0.8512183468705207, 0.03207403445393859]


Next, we aim to compare the performance of our approach to handling missing values against the approach of simply ignoring them. To achieve this, we will use the same test set for testing, and different training set. In one case, we will ignore the missing values, while in the other, we will use our approach to handle them. As result, we can see our approach to handling missing value has a much better performance than simply ignore it. 

In [49]:
##Train GBDT using dataset with missing value##
gb_ca = gradient_boosting(M=10,max_depth = 3,min_sample = 5,learning_rate = 1,method="binary classification",missing_value=True)
gb_ca.fit(X_train_ca_n,y_train_ca_n)

# performance evaluation
y_pred_ca_n_test = gb_ca.predict(X_test_ca)
print("GBDT performance on same test set (trained using dataset with missing value): ")
print("Accuracy: " + str(accuracy_score(y_test_ca, y_pred_ca_n_test)))
print("============= Classification Report =================")
print(classification_report(y_test_ca, y_pred_ca_n_test))

GBDT performance on same test set (trained using dataset with missing value): 
Accuracy: 0.8826530612244898
============= Classification Report =================
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       109
           1       0.89      0.84      0.86        87

    accuracy                           0.88       196
   macro avg       0.88      0.88      0.88       196
weighted avg       0.88      0.88      0.88       196



In [43]:
##Train GBDT using dataset removing missing value##
gb_ca = gradient_boosting(M=10,max_depth = 3,min_sample = 5,learning_rate = 1,method="binary classification",missing_value=True)
gb_ca.fit(X_train_ca,y_train_ca)

# performance evaluation
y_pred_ca_test = gb_ca.predict(X_test_ca)
print("GBDT performance on same test set (trained using dataset removing missing value): ")
print("Accuracy: " + str(accuracy_score(y_test_ca, y_pred_ca_test)))
print("============= Classification Report =================")
print(classification_report(y_test_ca, y_pred_ca_test))

GBDT performance on same test set (trained using dataset removing missing value) 
Accuracy: 0.8367346938775511
============= Classification Report =================
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       109
           1       0.80      0.85      0.82        87

    accuracy                           0.84       196
   macro avg       0.83      0.84      0.84       196
weighted avg       0.84      0.84      0.84       196



## Wine dataset (Multiclass classification)

I used this dataset to test the multiclass classification performance of our implemented GBDT. Our implementation of GBDT achieved comparable results to the GBDT implementation in scikit-learn, but was slower in terms of computation speed.

dateset reference: https://archive.ics.uci.edu/ml/datasets/wine+quality

In [8]:
# Data preprocessing
df_wine = pd.read_csv('D:\\stats790\\winequality-red.csv')

X = df_wine.drop('quality', axis=1)
y = df_wine['quality']

# normalize numerical predictors
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X))

# perform train-test split
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X, y, test_size=0.3, random_state=4)

In [18]:
##Find best parameter set (Hyperparameter tuning)##
parameters = {'M': [10,50,100], 
              'max_depth': [3,6], 
              'min_sample': [3,5],
              'learning_rate':[0.1,0.5,1]}
best_params_wine, best_score_wine = grid_search_cv_GBDT(parameters, X_train_wine,y_train_wine, method = "multiclass classification", kfold=2, scoring = "acc")
print(best_params_heart)
print(best_score_heart)

Current parameter set: {'M': 10, 'max_depth': 3, 'min_sample': 3, 'learning_rate': 0.1}
acc:0.5817547278303092
Current parameter set: {'M': 10, 'max_depth': 3, 'min_sample': 3, 'learning_rate': 0.5}
acc:0.5924706107845643
Current parameter set: {'M': 10, 'max_depth': 3, 'min_sample': 3, 'learning_rate': 1}
acc:0.5504871581906465
Current parameter set: {'M': 10, 'max_depth': 3, 'min_sample': 5, 'learning_rate': 0.1}
acc:0.5790729619217991
Current parameter set: {'M': 10, 'max_depth': 3, 'min_sample': 5, 'learning_rate': 0.5}
acc:0.5790458088423205
Current parameter set: {'M': 10, 'max_depth': 3, 'min_sample': 5, 'learning_rate': 1}
acc:0.557641196013289
Current parameter set: {'M': 10, 'max_depth': 6, 'min_sample': 3, 'learning_rate': 0.1}
acc:0.6022984283158701
Current parameter set: {'M': 10, 'max_depth': 6, 'min_sample': 3, 'learning_rate': 0.5}
acc:0.6041017122412471
Current parameter set: {'M': 10, 'max_depth': 6, 'min_sample': 3, 'learning_rate': 1}
acc:0.5817738947099411
Current 

In [13]:
##Prediction on test set##
gb_wine = gradient_boosting(M=100,max_depth = 3,min_sample = 5,learning_rate = 0.1,method="multiclass classification")
gb_wine.fit(X_train_wine,y_train_wine)

# performance evaluation
y_pred_wine_test = gb_wine.predict(X_test_wine)
print("Accuracy: " + str(accuracy_score(y_test_wine, y_pred_wine_test)))
print("============= Classification Report =================")
print(classification_report(y_test_wine, y_pred_wine_test))

Accuracy: 0.6333333333333333
============= Classification Report =================
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.14      0.07      0.09        15
           5       0.70      0.72      0.71       211
           6       0.59      0.64      0.62       190
           7       0.66      0.49      0.56        59
           8       0.00      0.00      0.00         3

    accuracy                           0.63       480
   macro avg       0.35      0.32      0.33       480
weighted avg       0.63      0.63      0.63       480



The following is the performance of GBDT from scikit learn, we can see the result is similar, but it is faster.

In [17]:
gbdt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=5)
gbdt.fit(X_train_wine, y_train_wine)
y_pred = gbdt.predict(X_test_wine)

print("Accuracy:", accuracy_score(y_test_wine, y_pred))
print("============= Classification Report =================")
print(classification_report(y_test_wine, y_pred))

Accuracy: 0.6375
============= Classification Report =================
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.12      0.07      0.09        15
           5       0.71      0.70      0.70       211
           6       0.61      0.68      0.64       190
           7       0.63      0.44      0.52        59
           8       0.50      0.33      0.40         3

    accuracy                           0.64       480
   macro avg       0.43      0.37      0.39       480
weighted avg       0.64      0.64      0.63       480

